In [20]:
from scipy.linalg import eig,eigh


import panel as pn, holoviews as hv, numpy as np
import param
pn.extension('katex','plotly')
hv.extension('bokeh')

from julia.api import Julia
jl = Julia(compiled_modules=False)
from julia import Main

import logging

# Suppress overly verbose logging
logging.disable(logging.CRITICAL)

%load_ext julia.magic

The julia.magic extension is already loaded. To reload it, use:
  %reload_ext julia.magic


In [2]:
%%julia
using Pkg
gla_dir = "../GenLinAlgProblems"
Pkg.activate(gla_dir)
using GenLinAlgProblems, LinearAlgebra, LaTeXStrings, Latexify, SymPy, Random

using PyCall
itikz = pyimport("itikz")
nM    = pyimport("itikz.nicematrix");

  Activating project at `C:\Users\jeff\NOTEBOOKS\elementary-linear-algebra\GenLinAlgProblems`


In [13]:
def stem(data, color='black', marker=True, label=None, hline=None):
    """Stem plot: data=(x, y, e). Draws vertical lines from y to e.
    Optionally adds a horizontal line at hline."""
    x, y, e = data
    vlines = [np.array([[x[i], y[i]], [x[i], e[i]]]) for i in range(len(x))]
    stems = hv.Path(vlines).opts(color=color, line_width=1.5)

    if marker:
        scatter = hv.Scatter((x, e)).opts(color=color, size=5)
        if label is not None:
            scatter = scatter.relabel(str(label))  # safely assign label
        stems *= scatter

    if hline is not None:
        stems *= hv.HLine(hline).opts(line_width=0.5, color='black')

    return stems

# 0.  Introduction

This notebook extends the study of the generalized eigenproblem beyond [**introductory mechanics**](EPE_intro.ipynb) and [**algorithmic formulation**](GEP_computation.ipynb).

The selected examples emphasize the structural ubiquity of $A x = \lambda B x$:  
$\qquad$ in coupled RLC circuits, the problem reappears as a resonance condition;  
$\qquad$ in multivariate statistics, it underlies canonical correlation and motivates the generalized singular value decomposition (GSVD).  

A numerical sensitivity experiment revisits conditioning issues introduced earlier, reinforcing the analytical–computational link.  
Collectively, these demonstrations consolidate the interpretation of the GEP as a unifying algebraic framework  
and provide a conceptual bridge toward the GSVD.

# 1  Revisiting the GEP Framework

In [4]:
pn.pane.LaTeX(r"""$\displaystyle A x = \lambda B x,\;\;$
with $A,B\in\mathbb{R}^{n\times n},\;
x\neq 0$""", styles={'font-size':'14pt'})

LaTeX(str, renderer='katex', styles={'font-size': '14pt'})

> **Reminder:**  
> When $B$ is positive definite, the problem defines a *weighted inner product*  
> $ \langle x,y\rangle_B = x^T B y $.  
> Eigenvectors for distinct eigenvalues are $B$-orthogonal.

# 2. Case Study: RLC Circuit Analogue

<div style="float:left;width:50%">

The resonance of coupled RLC circuits provides an electrical analogue  
to the mechanical vibration problem discussed earlier.  
Each branch of the circuit alternately stores energy in inductors and capacitors,  
leading to a system of equations that conserve total stored energy.

* The **magnetic energy** stored in the inductors is
$\;\; E_L = \tfrac{1}{2}\, i^T L\, i,$<br>
where $L$ is the inductance matrix and $i$ the vector of branch currents.
* The **electric energy** stored in the capacitors is
$\;\; E_C = \tfrac{1}{2}\, v^T C\, v = \tfrac{1}{2}\, i^T C^{-1}\, i,$<br>
since current and voltage are related by $i = C\,\dot{v}$.

Here $L$ is the inductance matrix and $C$ the capacitance matrix.  
Both $L$ and $C^{-1}$ weight the current $i$ in a manner
completely analogous to the kinetic and potential energies of a mechanical system.

</div><div style="float:left;width:50%;">
<img src="./Figs/RLCcircuit.svg" width=600></div>

____
When the circuit oscillates, energy transfers periodically between these two quadratic forms — magnetic (inductive) and electric (capacitive) —  
just as kinetic and potential energy exchange in a vibrating mechanical system.

Applying Kirchhoff’s and constitutive laws yields the coupled differential equation

$\qquad L\,\ddot{i} + C^{-1} i = 0.$

Seeking harmonic solutions $\;\;i(t) = \hat{i}\ e^{j\omega t}\;\;$ transforms this into

$\qquad (C^{-1} - \omega^2 L)\, \hat{i} = 0,$

which is the **generalized eigenvalue problem**

$\qquad \color{blue}\mathbf{C^{-1}\ \hat{i} = \omega^2\ L\,\ \hat{i}}.$

Here, $L$ and $C^{-1}$ serve as *energy metrics* — each defines a measure of stored energy in a different physical domain.  
Their interplay determines the natural frequencies $\omega$ and the modal current patterns $\hat{i}$.  

This explicit correspondence between energy balance and the generalized eigenproblem underscores how the same mathematical structure  
governs oscillations across physical systems—mechanical, electrical, and beyond.

**Remark:**
Both matrices in the circuit formulation are symmetric and positive definite under normal physical conditions:
* the inductance matrix $L$ is positive definite because each inductor stores positive magnetic energy $E_L = \tfrac{1}{2}, i^T L, i$,
* the inverse capacitance matrix $C^{-1}$ is positive definite because each capacitor stores positive electric energy $E_C = \tfrac{1}{2}, i^T C^{-1}, i$.

Consequently, the generalized eigenproblem $C^{-1} i = \omega^2 L, i$ is symmetric definite,<br>
ensuring that all resonance frequencies $\omega$ are real and positive and that the mode vectors are orthogonal with respect to $L$.

In [21]:
def circuit_modes(L, C, αL=0.1, αC=0.05):
    """
    Construct coupled inductance and capacitance matrices
    and solve the generalized eigenproblem:

        Lmat @ x = λ * Cmat @ x  ⇒  ω = sqrt(λ)

    Args:
        L (array-like): inductance values (H)
        C (array-like): capacitance values (F)
        αL (float): inductive coupling coefficient
        αC (float): capacitive coupling coefficient

    Returns:
        freqs (ndarray): resonance frequencies (rad/s)
        modes (ndarray): mode shapes (each column is a mode vector)
    """
    L = np.asarray(L, dtype=float)
    C = np.asarray(C, dtype=float)
    n = len(L)

    # Construct symmetric matrices
    Lmat = np.zeros((n, n))
    Cmat = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            if i == j:
                Lmat[i, j] = L[i]
                Cmat[i, j] = C[i]
            else:
                Lmat[i, j] = αL * np.sqrt(L[i] * L[j])
                Cmat[i, j] = αC * np.sqrt(C[i] * C[j])

    # Solve generalized eigenproblem: Lmat @ x = λ * Cmat @ x
    eigvals, eigvecs = eigh(Lmat, Cmat)
    eigvals = np.abs(eigvals)  # avoid small negatives from numerical errors
    ω = np.sqrt(eigvals)

    # Sort by frequency
    idx = np.argsort(ω)
    return ω[idx], eigvecs[:, idx]


In [22]:
def energy_partition(Lvals, Cvals, modes, freqs):
    """
    Compute energy fractions per mode with proper physical scaling:
    E_L = (1/2) ω² iᵀ L i
    E_C = (1/2)     iᵀ C⁻¹ i
    """
    L = np.diag(Lvals)
    Cinv = np.diag(1 / np.array(Cvals))
    energies = []
    for i, v in enumerate(modes.T):
        eL = freqs[i]**2 * v.T @ L @ v
        eC = v.T @ Cinv @ v
        e_tot = eL + eC
        energies.append((eL / e_tot, eC / e_tot))
    return np.array(energies)

#####################################################
class new_RLCViewer(pn.viewable.Viewer):
    # --- Parameters ---
    L1 = param.Number(default=1e-3,   bounds=(5e-4, 3e-3), step=1e-4)
    L2 = param.Number(default=2e-3,   bounds=(1e-3, 4e-3), step=1e-4)
    L3 = param.Number(default=1.5e-3, bounds=(5e-4, 3e-3), step=1e-4)
    C1 = param.Number(default=1e-6,   bounds=(5e-7, 2e-6), step=1e-7)
    C2 = param.Number(default=1.2e-6, bounds=(6e-7, 2.4e-6), step=1e-7)
    C3 = param.Number(default=0.8e-6, bounds=(4e-7, 1.6e-6), step=1e-7)
    αL = param.Number(default=0.1, bounds=(1e-12, 0.5), step=0.01)
    αC = param.Number(default=0.05, bounds=(1e-12, 0.3), step=0.01)

    def __init__(self, **params):
        super().__init__(**params)

        controls = pn.Column(
            "### RLC Parameters",
            pn.Row(self.param.L1, self.param.L2, self.param.L3),
            pn.Row(self.param.C1, self.param.C2, self.param.C3),
            pn.Row(self.param.αL, self.param.αC),
            pn.Spacer(height=8),
        )

        self._freq_pane   = pn.pane.HoloViews()
        self._branch_pane = pn.pane.HoloViews()

        top    = pn.Row(self._freq_pane, self._branch_pane)
        self._view = pn.Column(controls, pn.Column(top, sizing_mode="stretch_width"))

        self._update_plots()
        self.param.watch(self._update_plots, ['L1','L2','L3','C1','C2','C3','αL','αC'], onlychanged=True)
        pn.bind(self._update_plots, debounce=100)

    def view(self):
        return self._view

    def __panel__(self):
        return self.view()

    def _recompute(self):
        Lvals = np.array([self.L1, self.L2, self.L3], dtype=float)
        Cvals = np.array([self.C1, self.C2, self.C3], dtype=float)

        freqs, modes = circuit_modes(Lvals, Cvals, αL=self.αL, αC=self.αC)
        freqs = np.array(freqs, dtype=float)
        modes = np.array(modes, dtype=float)

        col_max = np.max(np.abs(modes), axis=0)
        col_max[col_max == 0] = 1.0
        modes_disp = modes / col_max[np.newaxis, :]

        e_frac = energy_partition(Lvals, Cvals, modes, freqs)
        return freqs, modes_disp, e_frac

    def _update_plots(self, *_):
        freqs, modes_disp, e_frac = self._recompute()
        xticks = [(1,"1"), (2,"2"), (3,"3")]

        # Frequencies in Hz
        freqs_Hz = freqs / (2 * np.pi)
        y_max = float(np.max(freqs_Hz) * 1.1) if freqs_Hz.size and np.max(freqs_Hz) > 0 else 1.0

        freq_plot = stem((np.arange(1, 4), np.zeros(3), freqs_Hz), color='navy').opts(
            title='Resonant Frequencies',
            xlabel='Mode index',
            ylabel='f (Hz)',
            width=260,
            height=260,
            xticks=xticks,
            ylim=(0, y_max),
            shared_axes=False,
            #framewise=True
        )
        self._freq_pane.object = freq_plot

        # Enhanced combined stem plot for branch amplitudes
        offsets = [-0.1, 0.0, 0.1]
        colors  = ['darkgreen', 'orange', 'purple']
        labels  = ['Branch 1', 'Branch 2', 'Branch 3']
        stems   = []

        for b in range(3):
            x = np.arange(1, 4) + offsets[b]
            y = np.zeros_like(x)
            e = modes_disp[b, :]
            stem_b = stem((x, y, e), color=colors[b], label=labels[b], hline=0)
            stems.append(stem_b)

        branch_plot = stems[0]
        for s in stems[1:]:
            branch_plot *= s

        branch_plot = branch_plot.opts(
            title='Branch Amplitudes per Mode',
            xlabel='Mode index',
            ylabel='Amplitude',
            width=500,
            height=260,
            xticks=xticks,
            ylim=(-1.1, 1.1),
            show_legend=True,
            legend_position='right',
            #tools=['hover'], toolbar='above',
            shared_axes=False #, framewise=True
        )
        self._branch_pane.object = branch_plot

rlc = new_RLCViewer()
rlc.servable()

Column
    [0] Column
        [0] Markdown(str)
        [1] Row
            [0] FloatSlider(end=0.003, name='L1', start=0.0005, step=0.0001, value=0.001)
            [1] FloatSlider(end=0.004, name='L2', start=0.001, step=0.0001, value=0.002)
            [2] FloatSlider(end=0.003, name='L3', start=0.0005, step=0.0001, value=0.0015)
        [2] Row
            [0] FloatSlider(end=2e-06, name='C1', start=5e-07, step=1e-07, value=1e-06)
            [1] FloatSlider(end=2.4e-06, name='C2', start=6e-07, step=1e-07, value=1.2e-06)
            [2] FloatSlider(end=1.6e-06, name='C3', start=4e-07, step=1e-07, value=8e-07)
        [3] Row
            [0] FloatSlider(end=0.5, name='ΑL', start=1e-12, step=0.01, value=0.1)
            [1] FloatSlider(end=0.3, name='ΑC', start=1e-12, step=0.01, value=0.05)
        [4] Spacer(height=8)
    [1] Column(sizing_mode='stretch_width')
        [0] Row
            [0] HoloViews(Overlay, height=260, sizing_mode='fixed', width=260)
            [1] HoloViews(Overlay, height=260, sizing_mode='fixed', width=500)

**Caption:**  
This figure displays the resonant frequencies (Hz) and corresponding modal current amplitudes for a three-branch coupled RLC circuit.  
The stem plot on the left shows the three natural frequencies,  
while the right plot overlays the normalized modal currents for each branch across the modes.  
Each mode represents a distinct pattern of current oscillation, influenced by the inductance and capacitance values  
as well as the coupling strengths ($\alpha_L$, $\alpha_C$).

As these parameters vary, the user should observe shifts in frequency and qualitative changes in mode shape  
— particularly the relative amplitude and phase (sign) of each branch within a mode.

# 3.  Case Study: Data-Analytic GEP and the Path to GSVD

In [7]:
rng=np.random.default_rng(1)
X=rng.normal(size=(100,3))
Y=X@np.diag([0.9,0.6,0.3])+0.1*rng.normal(size=(100,3))

Main.X,Main.Y = X,Y
Main.eval('''A = X' * Y; B = X' * X; C = Y' * Y; E = eigen(inv(B)*A*inv(C)*A')''')
vals = np.sqrt(np.maximum(np.array(Main.eval('E.values')),0))
hv.Bars((np.arange(1,4),vals)).opts(title='Canonical Correlations (√λ from GEP)',ylabel='Correlation')

:Bars   [x]   (y)

# 4.  Numerical Conditioning Demo

In [8]:
def sensitivity_demo(eps=1e-3):
    A=np.array([[3,1],[1,3]],float)
    B=np.array([[2,0],[0,1+eps]],float)
    Main.A,Main.B=A,B
    lam=np.array(Main.eval('eigen(A,B).values'))
    return lam

eps_vals=np.logspace(-3,-1,30)
lam_stack=np.array([sensitivity_demo(e) for e in eps_vals])
hv.Curve((eps_vals,lam_stack[:,0]),label='λ₁') * hv.Curve((eps_vals,lam_stack[:,1]),label='λ₂')

:Overlay
   .Curve.Λ_subscript_one :Curve   [x]   (y)
   .Curve.Λ_subscript_two :Curve   [x]   (y)

# 5.  Summary and Bridge to GSVD

<div style="float:left; padding-left:1.0cm;">

| Concept | Equation | Key Idea |
|:--|:--|:--|
| Standard EP | $A x = \lambda x$ | Unweighted modes |
| Generalized EP $\quad$ | $A x = \lambda B x \quad$ | Two metrics, $B$-orthogonality |
| GSVD | $A=U Σ_A X^T, B=V Σ_B X^T \qquad$ | Joint basis for paired subspaces |
</div>

- **Unifying view:** from RLC circuits to correlated datasets, the same algebraic structure governs modal relationships.
- **Numerical perspective:** well-conditioned B ensures stable eigenpairs.
- **Looking ahead:** the **GSVD** extends these ideas to non-square and rank-deficient pairs.